In [ ]:
# See ./README.md
import contextlib
import os
import sys
import time

import torch

import gemma.config
import gemma.model

DEVICE = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
if DEVICE == "mps":
    print("Can't use mps because https://github.com/pytorch/pytorch/issues/78044, upgrade to next version")
    DEVICE = "cpu"
print(f"Using {DEVICE}")

# Make it reproducible.
torch.manual_seed(12)

@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
    """Sets the default torch dtype to the given dtype."""
    torch.set_default_dtype(dtype)
    yield
    torch.set_default_dtype(torch.float)

def get_generator(seed):
    if DEVICE in ("cuda", "mps"):
        return torch.Generator(DEVICE).manual_seed(seed)
    return torch.Generator().manual_seed(seed)

def load_model(variant="2b", modelzip=None):
    # variant="7b"
    start = time.time()
    modelzip = modelzip or os.path.expanduser("~/Downloads/archive/gemma-2b.ckpt")
    if not os.path.isfile(modelzip):
        print("See ./README.md")
        sys.exit(1)

    cfg = gemma.config.get_model_config(variant)
    cfg.tokenizer = os.path.join(os.path.dirname(modelzip), "tokenizer.model")
    cfg.dtype = "float16" if DEVICE in ("cuda", "mps") else "float32"
    cfg.quant = False

    device = torch.device(DEVICE)
    with _set_default_tensor_type(cfg.get_dtype()):
        model = gemma.model.GemmaForCausalLM(cfg)
        model.load_weights(modelzip)
        model = model.to(DEVICE).eval()

    print(f"Model loaded in {time.time()-start:.1f}s")
    return model

def generate(p):
    start = time.time()
    result = model.generate(p, DEVICE)
    print(f"{result}\nin {time.time()-start:.1f}s")

In [ ]:
model = load_model(modelzip=os.path.expanduser("~/Téléchargements/archive/gemma-2b.ckpt"))

In [ ]:
generate("How are you doing?")